In [1]:
from datasetsforecast.m3 import M3

df = M3().load("../data", group="Monthly")[0]
df.sort_values(["unique_id", "ds"], inplace=True)
df.to_parquet("data/input/m3-monthly.parquet")

/home/pranav-pc/projects/ts/.venv/lib/python3.12/site-packages/datasetsforecast/m3.py:108: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq = pd.tseries.frequencies.to_offset(class_group.freq)


In [2]:
import json
import os

import pandas as pd
import torch
from tqdm import tqdm

# === Config ===
input_path = "data/input/m3-monthly.parquet"
output_path = "data/intermediate/m3-monthly_scaled.parquet"
scaler_save_path = "artifacts/m3-monthly_scalers.json"
scaler = "minmax"  # or "standard"
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using device: {device}")

# === Load Data ===
df = pd.read_parquet(input_path)
scaled_parts = []
scaler_stats = {}

# === Process per unique_id with tqdm ===
for uid, group in tqdm(df.groupby("unique_id"), desc="Scaling series"):
    y = torch.tensor(group["y"].values, dtype=torch.float32, device=device)

    if scaler == "minmax":
        min_val, max_val = y.min(), y.max()
        scaled_y = (y - min_val) / (max_val - min_val + 1e-8)

        scaler_stats[uid] = {
            "type": "minmax",
            "min": float(min_val.cpu()),
            "max": float(max_val.cpu()),
        }

    elif scaler == "standard":
        mean, std = y.mean(), y.std(unbiased=False)
        scaled_y = (y - mean) / (std + 1e-8)

        scaler_stats[uid] = {
            "type": "standard",
            "mean": float(mean.cpu()),
            "std": float(std.cpu()),
        }

    else:
        raise ValueError(f"Unsupported scaler type: {scaler}")

    group = group.copy()
    group["y_scaled"] = scaled_y.cpu().numpy()
    scaled_parts.append(group)

# === Save Scaled Data and Scaler Stats ===
scaled_df = pd.concat(scaled_parts).sort_values(["unique_id", "ds"])
scaled_df.to_parquet(output_path, index=False)

with open(scaler_save_path, "w") as f:
    json.dump(scaler_stats, f, indent=2)

print(f"✅ Scaled data saved to: {output_path}")
print(f"✅ Scaler stats saved to: {scaler_save_path}")

Using device: cuda


Scaling series:   0%|          | 0/1428 [00:00<?, ?it/s]

Scaling series:   0%|          | 1/1428 [00:00<03:11,  7.46it/s]

Scaling series:  32%|███▏      | 453/1428 [00:00<00:00, 2333.18it/s]

Scaling series:  64%|██████▎   | 908/1428 [00:00<00:00, 3270.63it/s]

Scaling series:  88%|████████▊ | 1260/1428 [00:00<00:00, 3127.67it/s]

Scaling series: 100%|██████████| 1428/1428 [00:00<00:00, 2880.93it/s]

✅ Scaled data saved to: data/intermediate/m3-monthly_scaled.parquet
✅ Scaler stats saved to: artifacts/m3-monthly_scalers.json
